# 🌐 Trust Network Evolution

*Watch trust propagate through multi-agent networks*

This notebook implements:
- Multi-agent trust networks based on established network science
- Trust propagation and cascade dynamics
- Network resilience analysis without claiming universal thresholds

Based on research from Barabási's network science and Watts' cascade models.

In [ ]:
# Install required packages
!pip install plotly pandas numpy ipywidgets networkx matplotlib -q

In [ ]:
#@title 🌐 Trust Network Evolution { display-mode: "form" }
#@markdown See how trust spreads (or collapses) through social networks!

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import networkx as nx
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown
from IPython.display import clear_output
import matplotlib.pyplot as plt

# Network color scheme
NETWORK_COLORS = {
    'trusted': '#2ECC71',       # Green
    'neutral': '#95A5A6',       # Gray
    'distrusted': '#E74C3C',    # Red
    'bridge': '#3498DB',        # Blue
    'isolated': '#34495E'       # Dark gray
}

class TrustNetwork:
    """Multi-agent trust network simulation based on network science principles"""
    def __init__(self, n_agents=20, topology='small_world', connectivity=0.3):
        """
        Initialize network with configurable topology.
        
        Args:
            n_agents: Number of agents in the network
            topology: Network structure ('random', 'small_world', 'scale_free')
            connectivity: Initial connection density
        """
        self.n_agents = n_agents
        self.agents = list(range(n_agents))
        
        # Create network structure based on topology
        if topology == 'random':
            self.graph = nx.erdos_renyi_graph(n_agents, connectivity)
        elif topology == 'small_world':
            k = max(2, int(n_agents * connectivity))
            self.graph = nx.watts_strogatz_graph(n_agents, k, 0.3)
        elif topology == 'scale_free':
            m = max(1, int(n_agents * connectivity / 2))
            self.graph = nx.barabasi_albert_graph(n_agents, m)
        else:
            self.graph = nx.erdos_renyi_graph(n_agents, connectivity)
        
        # Initialize trust matrix
        self.trust_matrix = np.zeros((n_agents, n_agents))
        for i, j in self.graph.edges():
            # Initial trust varies by topology
            if topology == 'small_world':
                initial_trust = np.random.uniform(0.4, 0.8)  # Higher initial trust
            else:
                initial_trust = np.random.uniform(0.3, 0.7)
            self.trust_matrix[i, j] = initial_trust
            self.trust_matrix[j, i] = initial_trust
            
        # Agent properties with heterogeneity
        self.trustworthiness = np.random.beta(5, 2, n_agents)  # Skewed toward trustworthy
        self.influence = np.random.power(2, n_agents)  # Power law distribution
        self.resilience = np.random.uniform(0.3, 0.8, n_agents)  # Trust recovery ability
        
        self.history = []
        self.cascade_threshold = 0.3  # Configurable cascade threshold
        
    def update_trust(self, interaction_prob=0.1, trust_decay=0.05, learning_rate=0.1):
        """One timestep of trust evolution with realistic dynamics"""
        new_trust = self.trust_matrix.copy()
        interactions_log = []
        
        # Random interactions
        for i in range(self.n_agents):
            if np.random.random() < interaction_prob:
                neighbors = list(self.graph.neighbors(i))
                if neighbors:
                    j = np.random.choice(neighbors)
                    
                    # Interaction outcome based on trustworthiness with noise
                    outcome_prob = self.trustworthiness[j] * (1 + np.random.normal(0, 0.1))
                    if np.random.random() < outcome_prob:
                        # Positive interaction
                        trust_increase = learning_rate * (1 - new_trust[i, j])
                        new_trust[i, j] = min(1, new_trust[i, j] + trust_increase)
                        interactions_log.append((i, j, 'positive'))
                    else:
                        # Negative interaction
                        trust_decrease = 2 * learning_rate  # Trust lost faster than gained
                        new_trust[i, j] = max(0, new_trust[i, j] - trust_decrease)
                        interactions_log.append((i, j, 'negative'))
        
        # Trust propagation (gossip/reputation effects)
        propagation_rate = 0.05
        for i in range(self.n_agents):
            neighbors = list(self.graph.neighbors(i))
            for j in neighbors:
                # Learn from neighbors' experiences
                common_neighbors = set(self.graph.neighbors(i)) & set(self.graph.neighbors(j))
                for k in common_neighbors:
                    # Influence-weighted learning
                    influence_factor = self.influence[j] * propagation_rate
                    trust_diff = self.trust_matrix[j, k] - self.trust_matrix[i, k]
                    new_trust[i, k] += influence_factor * trust_diff
                    new_trust[i, k] = np.clip(new_trust[i, k], 0, 1)
        
        # Natural trust decay with resilience factor
        decay_matrix = trust_decay * (1 - self.resilience)
        new_trust *= (1 - decay_matrix[:, np.newaxis])
        np.fill_diagonal(new_trust, 0)  # No self-trust
        
        self.trust_matrix = new_trust
        
        # Calculate network metrics
        trust_edges = [(i, j, {'weight': self.trust_matrix[i, j]}) 
                      for i, j in self.graph.edges() if self.trust_matrix[i, j] > 0]
        weighted_graph = nx.Graph()
        weighted_graph.add_nodes_from(range(self.n_agents))
        weighted_graph.add_edges_from(trust_edges)
        
        # Record state with comprehensive metrics
        active_trust = self.trust_matrix[self.trust_matrix > 0]
        self.history.append({
            'timestep': len(self.history),
            'avg_trust': np.mean(active_trust) if len(active_trust) > 0 else 0,
            'trust_variance': np.var(active_trust) if len(active_trust) > 0 else 0,
            'n_high_trust': np.sum(self.trust_matrix > 0.7),
            'n_low_trust': np.sum((self.trust_matrix > 0) & (self.trust_matrix < 0.3)),
            'clustering': nx.average_clustering(weighted_graph, weight='weight'),
            'largest_component': len(max(nx.connected_components(weighted_graph), key=len)) / self.n_agents,
            'interactions': interactions_log,
            'avg_degree': 2 * len(trust_edges) / self.n_agents if self.n_agents > 0 else 0
        })
        
    def add_crisis_event(self, crisis_type='targeted', intensity=0.5):
        """Simulate different types of trust crises"""
        if crisis_type == 'targeted':
            # Target high-influence agents
            n_affected = max(1, int(self.n_agents * 0.15))  # Affect 15% of network
            affected_agents = np.argsort(self.influence)[-n_affected:]
        elif crisis_type == 'random':
            # Random agents affected
            n_affected = max(1, int(self.n_agents * 0.2))
            affected_agents = np.random.choice(self.n_agents, n_affected, replace=False)
        elif crisis_type == 'cluster':
            # Affect a connected cluster
            start_node = np.random.choice(self.n_agents)
            affected_agents = list(nx.single_source_shortest_path_length(
                self.graph, start_node, cutoff=2).keys())[:int(self.n_agents * 0.25)]
        else:
            affected_agents = []
            
        # Apply trust damage
        for agent in affected_agents:
            self.trustworthiness[agent] *= (1 - intensity)
            # Immediate trust loss from all connections
            self.trust_matrix[:, agent] *= (1 - intensity * 0.8)
            self.trust_matrix[agent, :] *= (1 - intensity * 0.8)
            
        return affected_agents
            
    def visualize_network(self):
        """Create interactive network visualization"""
        # Use force-directed layout for better visualization
        pos = nx.spring_layout(self.graph, k=2/np.sqrt(self.n_agents), iterations=50)
        
        # Create edge traces colored by trust level
        edge_traces = []
        for i, j in self.graph.edges():
            trust_level = self.trust_matrix[i, j]
            
            if trust_level > 0.7:
                color = NETWORK_COLORS['trusted']
                width = 3
            elif trust_level > 0.3:
                color = NETWORK_COLORS['neutral']
                width = 2
            else:
                color = NETWORK_COLORS['distrusted']
                width = 1
                
            edge_trace = go.Scatter(
                x=[pos[i][0], pos[j][0], None],
                y=[pos[i][1], pos[j][1], None],
                mode='lines',
                line=dict(width=width, color=color),
                hoverinfo='none',
                showlegend=False
            )
            edge_traces.append(edge_trace)
        
        # Node trace with metrics
        node_x = [pos[i][0] for i in range(self.n_agents)]
        node_y = [pos[i][1] for i in range(self.n_agents)]
        
        # Calculate node metrics
        trust_received = np.sum(self.trust_matrix, axis=0)
        trust_given = np.sum(self.trust_matrix, axis=1)
        betweenness = nx.betweenness_centrality(self.graph)
        
        node_trace = go.Scatter(
            x=node_x,
            y=node_y,
            mode='markers+text',
            text=[str(i) for i in range(self.n_agents)],
            textposition="middle center",
            marker=dict(
                size=10 + trust_received * 3,
                color=self.trustworthiness,
                colorscale='Viridis',
                colorbar=dict(title="Trustworthiness"),
                line=dict(width=2, color='white')
            ),
            hovertemplate='Agent %{text}<br>' +
                         'Trustworthiness: %{marker.color:.2f}<br>' +
                         'Trust received: %{customdata[0]:.2f}<br>' +
                         'Trust given: %{customdata[1]:.2f}<br>' +
                         'Betweenness: %{customdata[2]:.3f}<br>' +
                         '<extra></extra>',
            customdata=list(zip(trust_received, trust_given, 
                              [betweenness[i] for i in range(self.n_agents)]))
        )
        
        # Create figure
        fig = go.Figure(data=edge_traces + [node_trace])
        
        fig.update_layout(
            title="Trust Network Visualization",
            showlegend=False,
            hovermode='closest',
            margin=dict(b=0, l=0, r=0, t=40),
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            plot_bgcolor='white',
            height=600
        )
        
        return fig
    
    def plot_evolution_metrics(self):
        """Plot trust evolution metrics without claiming universal thresholds"""
        df = pd.DataFrame(self.history)
        
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Average Trust Level', 'Trust Distribution',
                          'Network Connectivity', 'Network Health Indicators')
        )
        
        # Average trust with confidence interval
        fig.add_trace(
            go.Scatter(
                x=df['timestep'],
                y=df['avg_trust'],
                mode='lines',
                line=dict(color=NETWORK_COLORS['trusted'], width=3),
                name='Avg Trust'
            ),
            row=1, col=1
        )
        
        # Add variance band
        upper_bound = df['avg_trust'] + np.sqrt(df['trust_variance'])
        lower_bound = df['avg_trust'] - np.sqrt(df['trust_variance'])
        
        fig.add_trace(
            go.Scatter(
                x=df['timestep'].tolist() + df['timestep'].tolist()[::-1],
                y=upper_bound.tolist() + lower_bound.tolist()[::-1],
                fill='toself',
                fillcolor='rgba(78, 205, 196, 0.2)',
                line=dict(color='rgba(255,255,255,0)'),
                showlegend=False,
                name='Variance'
            ),
            row=1, col=1
        )
        
        # Trust distribution
        fig.add_trace(
            go.Scatter(
                x=df['timestep'],
                y=df['n_high_trust'],
                mode='lines',
                name='High Trust Edges',
                line=dict(color=NETWORK_COLORS['trusted'], width=2)
            ),
            row=1, col=2
        )
        
        fig.add_trace(
            go.Scatter(
                x=df['timestep'],
                y=df['n_low_trust'],
                mode='lines',
                name='Low Trust Edges',
                line=dict(color=NETWORK_COLORS['distrusted'], width=2)
            ),
            row=1, col=2
        )
        
        # Network connectivity
        fig.add_trace(
            go.Scatter(
                x=df['timestep'],
                y=df['largest_component'],
                mode='lines',
                fill='tozeroy',
                line=dict(color=NETWORK_COLORS['bridge'], width=2),
                name='Largest Component'
            ),
            row=2, col=1
        )
        
        # Multiple health indicators
        fig.add_trace(
            go.Scatter(
                x=df['timestep'],
                y=df['clustering'],
                mode='lines',
                line=dict(color=NETWORK_COLORS['isolated'], width=2),
                name='Clustering',
                yaxis='y2'
            ),
            row=2, col=2
        )
        
        fig.add_trace(
            go.Scatter(
                x=df['timestep'],
                y=df['avg_degree'] / df['avg_degree'].iloc[0] if len(df) > 0 else [],
                mode='lines',
                line=dict(color=NETWORK_COLORS['neutral'], width=2, dash='dash'),
                name='Relative Connectivity'
            ),
            row=2, col=2
        )
        
        fig.update_layout(
            height=700,
            showlegend=True,
            title_text="Trust Network Evolution Metrics"
        )
        
        return fig

# Initialize network
network = TrustNetwork(n_agents=20, topology='small_world')

# Interactive simulation
@interact(
    timesteps=IntSlider(min=10, max=200, step=10, value=50, description='Timesteps:'),
    interaction_prob=FloatSlider(min=0.01, max=0.5, step=0.01, value=0.1, 
                                description='Interaction Rate:'),
    trust_decay=FloatSlider(min=0, max=0.1, step=0.01, value=0.02,
                           description='Trust Decay:'),
    topology=Dropdown(options=['random', 'small_world', 'scale_free'],
                     value='small_world', description='Topology:'),
    crisis_timestep=IntSlider(min=0, max=100, step=10, value=0,
                             description='Crisis at:'),
    crisis_type=Dropdown(options=['none', 'targeted', 'random', 'cluster'],
                        value='none', description='Crisis Type:'),
    crisis_intensity=FloatSlider(min=0, max=1, step=0.1, value=0.5,
                                description='Crisis Intensity:')
)
def run_network_simulation(timesteps, interaction_prob, trust_decay, topology,
                         crisis_timestep, crisis_type, crisis_intensity):
    # Reset network with new topology
    global network
    network = TrustNetwork(n_agents=20, topology=topology)
    
    # Run simulation
    for t in range(timesteps):
        network.update_trust(interaction_prob, trust_decay)
        
        # Crisis event
        if crisis_type != 'none' and crisis_timestep > 0 and t == crisis_timestep:
            affected = network.add_crisis_event(crisis_type, crisis_intensity)
            print(f"💥 {crisis_type.title()} crisis at timestep {t}!")
            print(f"   Affected {len(affected)} agents: {affected[:5]}{'...' if len(affected) > 5 else ''}")
            print(f"   Intensity: {crisis_intensity:.1%} trust damage")
    
    # Visualizations
    if network.history:
        final_state = network.history[-1]
        print(f"\n📊 Simulation complete! Timesteps: {timesteps}")
        print(f"Network topology: {topology}")
        print(f"Final average trust: {final_state['avg_trust']:.3f}")
    else:
        print("No simulation data generated.")
        return
    
    # Network visualization
    network_fig = network.visualize_network()
    network_fig.show()
    
    # Evolution metrics
    metrics_fig = network.plot_evolution_metrics()
    metrics_fig.show()
    
    # Network analysis
    print(f"\n📊 Network Analysis:")
    print(f"Average Trust: {final_state['avg_trust']:.3f}")
    print(f"Trust Variance: {final_state['trust_variance']:.3f}")
    print(f"Network Clustering: {final_state['clustering']:.3f}")
    print(f"Largest Component: {final_state['largest_component']:.1%} of network")
    print(f"High Trust Connections: {final_state['n_high_trust']}")
    print(f"Low Trust Connections: {final_state['n_low_trust']}")
    
    # Trust leaders and isolates
    trust_received = np.sum(network.trust_matrix, axis=0)
    leaders = np.argsort(trust_received)[-3:]
    isolates = np.where(trust_received < 0.1)[0]
    
    print(f"\n🌟 Most Trusted Agents: {leaders[::-1].tolist()}")
    if len(isolates) > 0:
        print(f"🚫 Isolated Agents: {isolates.tolist()}")
    
    # Resilience insights without claiming universal thresholds
    print(f"\n💡 Network Insights:")
    if final_state['clustering'] > 0.5:
        print("- High clustering suggests strong local trust communities")
    elif final_state['clustering'] < 0.2:
        print("- Low clustering indicates dispersed trust patterns")
        
    if crisis_type != 'none' and crisis_timestep > 0:
        pre_crisis_trust = network.history[crisis_timestep-1]['avg_trust'] if crisis_timestep > 0 else 0.5
        recovery_rate = (final_state['avg_trust'] - network.history[crisis_timestep+1]['avg_trust']) / (
            timesteps - crisis_timestep - 1) if timesteps > crisis_timestep + 1 else 0
        print(f"\n📈 Crisis Recovery:")
        print(f"- Pre-crisis trust: {pre_crisis_trust:.3f}")
        print(f"- Recovery rate: {recovery_rate:.4f} per timestep")
        print(f"- Network showed {'resilience' if final_state['avg_trust'] > 0.5 * pre_crisis_trust else 'vulnerability'} to {crisis_type} attack")

#@markdown ---
#@markdown ### 🔬 Network Experiments:
#@markdown 1. **Topology Comparison**: How do different network structures handle trust?
#@markdown 2. **Crisis Recovery**: Compare recovery from different types of attacks
#@markdown 3. **Parameter Sensitivity**: Find optimal decay/interaction rates for your network
#@markdown 4. **Cascading Failures**: When do local failures become global?

## 🌐 Understanding Trust Networks

This simulation explores how trust propagates through multi-agent networks based on established network science principles.

### Network Topologies
- **Random (Erdős-Rényi)**: Connections form randomly with equal probability
- **Small World (Watts-Strogatz)**: High clustering with short path lengths
- **Scale-Free (Barabási-Albert)**: Power-law degree distribution with hubs

### Trust Dynamics
- **Direct interactions**: Trust changes through personal experience
- **Reputation effects**: Trust propagates through shared connections
- **Asymmetric updates**: Trust is lost faster than gained (empirically validated)

### Crisis Types
- **Targeted**: Attacks high-influence nodes (hubs)
- **Random**: Affects random agents
- **Cluster**: Damages a connected region

### Key Insights
- Network structure significantly affects trust resilience
- Recovery patterns depend on topology and crisis type
- No universal thresholds - optimal parameters are context-dependent

The clustering coefficient indicates local trust density but doesn't determine global resilience alone.